In [1]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import numpy as np
import win32com.client
import os
# VERSION 3.00
import re
#VERSION  3.01
import datetime
""""
- ADDED 'workbook.close()'  AND 'o.Workbooks.Close()' TO MAKE SURE THAT THE MEMORY GETS REUSED 
- IN ADDITION TO THAT, IF FOR SOME REASON THE PROCESS STOPS (I.E. STATES THAT ONE OF THE EXCEL FILES IS OPEN), CONSIDER THE
 FOLLOWING:
      - THE PDF COULD NOT FIND THE DESTINATION FOLDER (MAKE SURE THE DESTINATION FOLDER IS BEING CREATED).

"""

# VERSION 3.02
"""
- FIXED THE BORDER LINES IN THE EXCEL FILES
- UPDATE ON THE EXCEL TEMPLATE (V2)
"""

'\n- FIXED THE BORDER LINES IN THE EXCEL FILES\n- UPDATE ON THE EXCEL TEMPLATE (V2)\n'

In [2]:
# DATA SOURCE (WHERE YOU ARE GETTING ALL THE DATA SCORES)

xl = pd.ExcelFile("N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\\FY2019 Priority List Workbook Final - Full.xlsx")

pd.set_option('display.max_columns', 500)
xl.sheet_names

df = xl.parse("PSH Scoring",header=[0,1,2])
           

In [3]:
df.head(2)

Source is APR Data filtered to only PSH Source is APR Data filtered to only PSH  \
**Only APR Grant number Needed                   **Only APR Grant number Needed   
Project Number                                                      APR Grant #   
CA0331                                                          CA0331L9D001609   
CA0356                                                          CA0356L9D001608   

Source is APR Data filtered to only PSH                            \
**Only APR Grant number Needed                                      
Project Number                          Consolidated FileShare ID   
CA0331                                        CA0331       LA_ACF   
CA0356                                           NaN       LA_SRP   

Source is APR Data filtered to only PSH                                    \
**Only APR Grant number Needed                                              
Project Number                                                    Grantee   
CA0331                                             A Community of Friends   
CA0356                                   Alliance for Housing and Healing   

Source is APR Data filtered to only PSH                                    \
**Only APR Grant number Needed                                              
Project Number                                                     Agency   
CA0331                                             A Community of Friends   
CA0356                                   Alliance for Housing and Healing   

Source is APR Data filtered to only PSH                             \
**Only APR Grant number Needed                                       
Project Number                                             Project   
CA0331                                   CA0331 39 West Apartments   
CA0356                                         CA0356 CHOISS SPA 2   

Source is APR Data filtered to only PSH                                    \
**Only APR Grant number Needed                                              
Project Number                                                 Population   
CA0331                                                                NaN   
CA0356                                   Chronic Homeless, Mental Illness   

Source is APR Data filtered to only PSH                           Total Score  \
**Only APR Grant number Needed                             Unnamed: 8_level_1   
Project Number                          Cost Effectiveness        Prop. Score   
CA0331                                                 NaN           0.871429   
CA0356                                                 NaN           0.882353   

Source is APR Data filtered to only PSH                     \
**Only APR Grant number Needed          Unnamed: 9_level_1   
Project Number                               Points Earned   
CA0331                                           87.142857   
CA0356                                           75.000000   

Source is APR Data filtered to only PSH                      \
**Only APR Grant number Needed          Unnamed: 10_level_1   
Project Number                              Points Possible   
CA0331                                                  100   
CA0356                                                   85   

Source is APR Data filtered to only PSH Scoring Area 1: HMIS Participation  \
**Only APR Grant number Needed                      Participation (APR Q6)   
Project Number                                                    Response   
CA0331                                                                 Yes   
CA0356                                                                 Yes   

Source is APR Data filtered to only PSH                                \
**Only APR Grant number Needed                                          
Project Number                          Points Earned Points Possible   
CA0331                                              5               5   
CA0356       

In [4]:
#df2["Source is APR Data filtered to only PSH",'**Only APR Grant number Needed']

In [4]:
def update_range(worksheet, data, cell_range=None, named_range=None):
     """
     Updates an excel worksheet with the given data.
     :param worksheet: an excel worksheet
     :param data: data used to update the worksheet cell range (list, tuple, np.ndarray, pd.Dataframe)
     :param cell_range: a string representing the cell range, e.g. 'AB12:XX23'
     :param named_range: a string representing an excel named range
     """
 
     def clean_data(data):
         if not isinstance(data, (list, tuple, np.ndarray, pd.DataFrame)):
             raise TypeError('Invalid data, data should be an array type iterable.')
 
         if not len(data):
             raise ValueError('You need to provide data to update the cells')
 
         if isinstance(data, pd.DataFrame):
             data = data.values
 
         elif isinstance(data, (list, tuple)):
             data = np.array(data)
 
         return np.hstack(data)
 
     def clean_cells(worksheet, cell_range, named_range):
         # check that we can access a cell range
         if not any((cell_range, named_range) or all((cell_range, named_range))):
             raise ValueError('`cell_range` or `named_range` should be provided.')
 
         # get the cell range
         if cell_range:
             try:
                 cells = np.hstack(worksheet[cell_range])
             except (CellCoordinatesException, AttributeError):
                 raise ValueError('The cell range provided is invalid, cell range must be in the form XX--[:YY--]')
 
         else:
             try:
                 cells = worksheet.get_named_range(named_range)
             except (NamedRangeException, TypeError):
                 raise ValueError('The current worksheet {} does not contain any named range {}.'.format(
                     worksheet.title,
                     named_range))
 
         # checking that we have cells to update, and data
         if not len(cells):
             raise ValueError('You need to provide cells to update.')
 
         return cells
 
     cells = clean_cells(worksheet, cell_range, named_range)
     data = clean_data(data)
 
     # check that the data has the same dimension as cells
     if len(cells) != data.size:
         raise ValueError('Cells({}) should have the same dimension as the data({}).'.format(len(cells), data.size))
 
     for i, cell in enumerate(cells):
         cell.value = data[i]
            
            


In [5]:
# 


m = df['Source is APR Data filtered to only PSH','**Only APR Grant number Needed','Grantee'].str.contains('HOUSING AUTHORITY of The City of Los Angeles | of The City of Los Angeles', flags=re.IGNORECASE) \
    | df['Source is APR Data filtered to only PSH','**Only APR Grant number Needed','Grantee'].str.contains('HOUSING AUTHORITY OF THE CITY OF LA', flags=re.IGNORECASE)         \
    | df['Source is APR Data filtered to only PSH','**Only APR Grant number Needed','Grantee'].str.contains('Housing of the City of Los Angeles', flags=re.IGNORECASE)         



df['Source is APR Data filtered to only PSH','**Only APR Grant number Needed','Grantee (Updated)'] = np.where(m, "Housing Authority of The City of Los Angeles",df['Source is APR Data filtered to only PSH','**Only APR Grant number Needed','Grantee'])
df['Source is APR Data filtered to only PSH','**Only APR Grant number Needed','Grantee (Updated)'] =df['Source is APR Data filtered to only PSH','**Only APR Grant number Needed','Grantee (Updated)'].str.strip()

In [6]:
# CREATING FOLDERS BASED ON GRANTEE NAMES
# IMPORTANT : ONLY RUN THESE CELL IF YOU WOULD LIKE TO CREATE FOLDERS inside the PHA filder (WHETHER IS PRELIMINARY OR FINAL)


for index,row in df.iterrows():
    #k = row['Grantee']
    #G = df[df['FileShare ID'].str.contains(k,regex=False)]
    k = row['Source is APR Data filtered to only PSH','**Only APR Grant number Needed','Grantee (Updated)']
    BasePath =  r'N:\\Outcomes\\CoC Performance Evaluation Process\\FY2019 NOFA Performance Evaluation Documents\\Priority Listing Working Documents\\Evaluation Reports\\Final Score Reports\\PHA\\' + str(k)
    #if not os.path.exists(os.path.join(BasePath,k)):
        #os.makedirs(os.path.join(BasePath,k))
    if not os.path.exists(BasePath):
            os.makedirs(BasePath) 



In [6]:
# CREATING THE FOLDER IN THE FILESHARE ID DRIVE
# IMPORTANT : ONLY RUN THESE CELL IF YOU WOULD LIKE TO CREATE FOLDERS FOR THE FILESHARE ID (WHETHER IS PRELIMINARY OR FINAL)

for index,row in df.iterrows():

    #newpath = r'C:\Users\felixa\Documents\Felix\test'
    k = row['Source is APR Data filtered to only PSH','**Only APR Grant number Needed','FileShare ID']
    newpath = r'Q:\\'  + str(k) + '\\Continuum of Care NOFA Renewals\\NOFA 2019\\Renewal Evaluations\\Final Score Reports'
    if not os.path.exists(newpath):
            os.makedirs(newpath) 


In [8]:
#df['Housing First'].head()

In [7]:
# In order to rn this report YOU NEED THE FUNCTION ABOVE



for index,row in df.iterrows():
    
    k = row['Source is APR Data filtered to only PSH','**Only APR Grant number Needed','APR Grant #']
    GrantN = df[df['Source is APR Data filtered to only PSH','**Only APR Grant number Needed','APR Grant #'].str.contains(k, na = False)]
    #Adding the Values
    
    
    
    VALUES1  = pd.DataFrame(index=['Housing Stability', 'Served HN Populations', 'Minimize Negative Turnover',
                               'Maintain or Increase Income Overall', 'Bed/Unit Utilization', 'Commitment to Policy Priorities',
                               'Spend Down', 'Data Quality', 'HMIS Participation'],
                         columns=['Points Eearned', 'Points Possible'])
    VALUES1.iloc[0] = GrantN['Performance','Housing Stability/Retention (Q27-29)'][['Points Earned','Points Possible']].values
    VALUES1.iloc[1] = GrantN['Scoring Area 4: Serve High Need Population','Sub Total'][['Points Earned','Points Possible']].values
    VALUES1.iloc[2] = GrantN['Performance','Minimize Negative Turnover (Q27-29)'][['Points Earned','Points Possible']].values
    VALUES1.iloc[3] = GrantN['Performance','Maintain/Increase Income (APR Q24b3)'][['Points Earned','Points Possible']].values
    VALUES1.iloc[4] = GrantN['Scoring Area 2: Bed Utilization','SubTotal'][['Points Earned','Points Possible']].values
    VALUES1.iloc[5] = GrantN['Housing First','SubTotal'][['Points Earned','Points Possible']].values
    VALUES1.iloc[6] = GrantN['Scoring Area 3: Spend-Down','Sub Total'][['Points Earned','Points Possible']].values
    VALUES1.iloc[7] = GrantN['Scoring Area 1: HMIS Participation','Data Quality (APR Q7)'][['Points Earned','Points Possible']].values
    VALUES1.iloc[8] = GrantN['Scoring Area 1: HMIS Participation','Participation (APR Q6)'][['Points Earned','Points Possible']].values
    
    VALUES1['APR Result']  =    [GrantN['Performance','Housing Stability/Retention (Q27-29)']['Rate'][0],
                            GrantN['Scoring Area 4: Serve High Need Population','CES Referals']['Rate'][0],
                            GrantN['Performance','Minimize Negative Turnover (Q27-29)']['Rate'][0],
                            GrantN['Performance','Maintain/Increase Income (APR Q24b3)']['Rate'][0],
                            GrantN['Scoring Area 2: Bed Utilization','Unit Utilization (APR8-11)']['Rate'][0],
                            GrantN['Housing First','Unnamed: 47_level_1']['Response'][0],
                            GrantN['Scoring Area 3: Spend-Down','Spend-Down Rate (Q31a4)']['Rate'][0],
                            GrantN['Scoring Area 1: HMIS Participation','Data Quality (APR Q7)']['Rate'][0],
                            GrantN['Scoring Area 1: HMIS Participation','Participation (APR Q6)']['Response'][0]]
             
   
                              
                              
                            
    VALUES1 = VALUES1[['APR Result', 'Points Eearned', 'Points Possible']]

    #--------------------
        
    VALUES2 = pd.DataFrame(index = ['Grant Number (APR #)', 'Grant Number', 'Grantee Name', \
                               'Operating Agency', 'Project Name', 'Project Type', 'Target Population', 'FileShare ID']
                      , columns=['values'])

    VALUES2.iloc[0] = GrantN['Source is APR Data filtered to only PSH','**Only APR Grant number Needed']['APR Grant #'].values
    VALUES2.iloc[1] = GrantN['Source is APR Data filtered to only PSH','**Only APR Grant number Needed']['APR Grant #'].str[0:6].values
    VALUES2.iloc[2] = GrantN['Source is APR Data filtered to only PSH','**Only APR Grant number Needed']['Grantee (Updated)'].values
    VALUES2.iloc[3] = GrantN['Source is APR Data filtered to only PSH','**Only APR Grant number Needed']['Agency'].values
    VALUES2.iloc[4] = GrantN['Source is APR Data filtered to only PSH','**Only APR Grant number Needed']['Project'].values
        # STILL NEEDS TO ADD PROJECT TYPE
    VALUES2.iloc[5] = 'PSH'#THIS COULD BE A PARAMETER! FOR OUR FUTURE FUNCTION 
    VALUES2.iloc[6] = GrantN['Source is APR Data filtered to only PSH','**Only APR Grant number Needed']['Population'].values
    
    # ADDING THE FILESHARE_ID: (ADDED ON 7/25/2019)
    VALUES2.iloc[7] = GrantN['Source is APR Data filtered to only PSH','**Only APR Grant number Needed']['FileShare ID'].values
    
    
    VALUES2['empty1'] = np.nan
    VALUES2['empty2'] = np.nan
    #VALUES2['empty3'] = np.nan    
    VALUES2 = VALUES2[['empty1','values','empty2']]    
        #-----------
    VALUES3 = pd.DataFrame(index= ['Score Result'], columns =['Total Points Earned', 'Total Points Possible','Project Final Score'])
    VALUES3.iloc[0] = [VALUES1['Points Eearned'].sum(),VALUES1['Points Possible'].sum() ,VALUES1['Points Eearned'].sum()/VALUES1['Points Possible'].sum() ]
    VALUES3[['Total Points Earned', 'Total Points Possible']]
    
    # ----------EXCEL GENERATION----------
    
    #---TEMPLATE SOURCE (THE COMMENTED BELOW ALSO WORK, BUT THE SECOND ONE IS BETTER IF YOU DECIDE TO CHOOSE A TEMPLE FROM A DIFFERENT LOCATION)
    #workbook = load_workbook('Template.xlsx')
    
    
    workbook = load_workbook('N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\Priority Listing Working Documents\Evaluation Reports\Final Score Reports\PSH\Python\Template - PSH v2.xlsx')
    
    
    worksheet = workbook.active
    worksheet.title = 'template'
    update_range(workbook.active, VALUES1.values, named_range='VALUES1')
    update_range(workbook.active, VALUES2.values, named_range='VALUES2')

    #total points earned
    update_range(workbook.active, VALUES3[['Total Points Earned', 'Total Points Possible']].values, named_range='VALUES3')


    #Project Final Score
    update_range(workbook.active, VALUES3['Project Final Score'].values, named_range='VALUES4')
    
    workbook.save(str(k) + '.xlsx')
    workbook.close() # ADDED
    #-----------------------------------------------
    
    
    
    #-------------PDF CONVERSION----------------------------
    
    o = win32com.client.Dispatch("Excel.Application")
    o.Visible = False

    # LOCATION OF ALL THE EXCEL FILES-----
    
    
    wb_path =  r'N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\Priority Listing Working Documents\Evaluation Reports\Final Score Reports\PSH\Python\\' +  str(k)  + '.xlsx'

    wb = o.Workbooks.Open(wb_path) # OPENING EACH FILE (STILL WONDERING IF WE NEED THIS PIECE, SOMETHING TO CONSIDER IN THE FUTURE)

    ws_index_list = [1] #say you want to print these sheets
    
     # ----PATH TO THE FILESHARE ID -----
    # MAKE SURE YOU HAVE CREATED ALL THE FOLDERS BEFORE SENDING, OTHERWISE YOUR EXCEL FILES MIGHT GET STUCK 
    # TRYING TO FIND A DESTINATION FOLDER THAT DOESNT EXIST.
    
    path_to_pdf = 'Q:\\' + GrantN['Source is APR Data filtered to only PSH','**Only APR Grant number Needed']['FileShare ID'] + '\Continuum of Care NOFA Renewals\\NOFA 2019\\Renewal Evaluations\\Final Score Reports\\' + str(k) + '.pdf'
    print(path_to_pdf)
    #-----------------------------------
    
    #-------- PATH TO A PROJECT TYPE (N DRIVE, BROKEN DOWN BY PROJECT TYPE )------------------------
    #path_to_pdf = 'N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\Priority Listing Working Documents\Evaluation Reports\Final Score Reports\PSH\PDF\\' +  str(k)  + '.pdf'
    #print(path_to_pdf)
    #-------------------------------------------------------------------------------
    
    #wb.WorkSheets(ws_index_list).Select() # THIS WAS COMMENTED IN 8/21/2019

    
    wb.ActiveSheet.ExportAsFixedFormat(0, path_to_pdf)
    o.Workbooks.Close()

d = datetime.datetime.today()
print("FINAL REPORTS FOR PSH HAVE BEEN COMPLETED! :). Date and Time :" ,d.strftime("%d-%B-%Y %H:%M:%S") )      

C:\Users\felixa\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2819: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)
C:\Users\felixa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: Call to deprecated function get_named_range (Ranges are workbook objects. Use wb.defined_names[range_name]).


CA0331    Q:\LA_ACF\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0356    Q:\LA_SRP\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0357    Q:\LA_SRP\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA1342    Q:\LA_DMH\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0917    Q:\LA_ACF\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0783    Q:\LA_ACF\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA1220    Q:\LA_ACF\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0916    Q:\LA_ACF\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0335    Q:\LA_ACF\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0336    Q:\LA_ACF\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA1594    Q:\LA_ALC\Continuum of Care NOFA Renewals\NOFA...


CA0359    Q:\LA_ACC\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0401    Q:\LA_SUO\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA1104    Q:\LA_ACF\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0860    Q:\LA_HSW\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0365    Q:\LA_DMH\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA1046    Q:\LA_DMH\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA1158    Q:\LA_DMH\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0915    Q:\LA_ACF\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA1157    Q:\LA_ACF\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0914    Q:\LA_AAD\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0421    Q:\LA_NDI\Continuum of Care NOFA Renewals\NOFA...


In [18]:
#df['Source is APR Data filtered to only PSH','**Only APR Grant number Needed','Grantee'].iloc[0]

In [ ]:
#TO EXPORT THE FILES BY GRANTEEE NAME (STILL UNDER REVIEW)

for index,row in df.iterrows():
    
    k = row['Source is APR Data filtered to only PSH','**Only APR Grant number Needed','APR Grant #']
    GrantN = df[df['Source is APR Data filtered to only PSH','**Only APR Grant number Needed','APR Grant #'].str.contains(k, na = False)]
   

    #-------------PDF CONVERSION----------------------------
    o = win32com.client.Dispatch("Excel.Application")
    #o.Interactive = False
    o.Visible = False
    
    
    # LOCATION OF ALL THE EXCEL FILES------
    wb_path =  r'N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\Priority Listing Working Documents\Evaluation Reports\Final Score Reports\PSH\Python\\' +  str(k)  + '.xlsx'

    wb = o.Workbooks.Open(wb_path) # OPENING EACH FILE (STILL WONDERING IF WE NEED THIS PIECE, SOMETHING TO CONSIDER IN THE FUTURE)

    ws_index_list = [1] #say you want to print these sheets
    
    
    # -------PATH TO PHA (N DRIVE, BROKEN DOWN BY GRANTEE NAME)----------------------
   
    # MAKE SURE YOU HAVE CREATED ALL THE FOLDERS BEFORE SENDING, OTHERWISE YOUR EXCEL FILE MIGHT GET STUCK
    # TRYING TO FIND A DESTINATION FOLDER THAT DOES NOT EXIST
    
    path_to_pdf = 'N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\Priority Listing Working Documents\Evaluation Reports\Final Score Reports\PHA\\' +  GrantN['Source is APR Data filtered to only PSH','**Only APR Grant number Needed']['Grantee (Updated)']  + '\\'  +  str(k)  + '.pdf'
    print(path_to_pdf)
    #print(os.getcwd())
  
    #wb.WorkSheets(ws_index_list).Select()

    
    wb.ActiveSheet.ExportAsFixedFormat(0, path_to_pdf)
    
    
    o.Workbooks.Close() # ADDED 
    #os.chdir('N:\\Outcomes\\CoC Performance Evaluation Process\\FY2019 NOFA Performance Evaluation Documents\\Priority Listing Working Documents\\Evaluation Reports\\Final Score Reports\\PSH\\Python')
    

In [96]:
# CREATING AN EXCEL FILE WITH THE UPDATED GRANTEE NAMES

K = df['Source is APR Data filtered to only PSH','**Only APR Grant number Needed'][['Grantee','Grantee (Updated)']]
K.to_excel("NOFA 2019 Updated Grantee Names.xlsx")

In [141]:

import win32api
win32api.FormatMessage(-2147024809)

'The parameter is incorrect.\r\n'